In [2]:

import pandas as pd
df = pd.read_csv('./DataSources/k_means_with_5_features.csv') #where the K=4
df_cluster_0  = df[df['Cluster'] == 0]
df_cluster_0


Index(['Conversion ID', 'Advertiser', 'Campaign', 'Conversion Status',
       'Order ID', 'Conversion Time', 'Click Time', 'Timezone', 'Currency',
       'Original Order Value', 'Country Code', 'Device Type', 'ex3/category',
       'Cust Type', 'UID', 'Year', 'Month', 'Recency', 'Frequency',
       'Total Spend by Customer', 'Total Spend by Customer (All Advertisers)',
       'Wallet Share', 'Wallet Share by Merchant%', 'Total Spend in Category',
       'Total Spend by Customer in Category', 'Category Wallet Share',
       'Wallet Share by category %', 'Cluster'],
      dtype='object')

In [9]:
df = df_cluster_0
# Normalize your metrics (You might want to adjust the normalization method)
df['Recency_Score'] = 1 / (1 + df['Recency'])  # Assuming lower recency is better
df['Frequency_Score'] = (df['Frequency'] - df['Frequency'].min()) / (df['Frequency'].max() - df['Frequency'].min())
#['Spend_Score'] = (df['Total Spend by Customer'] - df['Total Spend by Customer'].min()) / (df['Total Spend by Customer'].max() - df['Total Spend by Customer'].min())
df['Wallet_Share_Merchant_Score'] = df['Wallet Share by Merchant%'] / 100  # Assuming it's in percent and normalizing
df['Wallet_Share_Category_Score'] = df['Wallet Share by category %'] / 100  # Assuming it's in percent and normalizing

# Composite Score - Feel free to adjust weights
df['Value_Score'] = (df['Recency_Score'] * 0.2) + (df['Frequency_Score'] * 0.2)  + \
                    (df['Wallet_Share_Merchant_Score'] * 0.2) + (df['Wallet_Share_Category_Score'] * 0.2)
                    
df['ABC_Category'] = pd.qcut(df['Value_Score'], [0, 0.5, 0.8, 1.0], labels=['C', 'B', 'A'])


In [18]:
cluster_0_label_A_categories = df[df['ABC_Category'] == 'A']['ex3/category'].value_counts().head(10)
cluster_0_label_A_categories

ex3/category
Groceries                     95
Home & Living                 66
Food & Beverage               60
Beauty                        36
Fashion Accessories           28
Women's Shoes and Clothing    24
Women Clothes                 19
Women Bags                    16
Hobbies & Collections         16
Household Supplies            15
Name: count, dtype: int64

In [20]:
df = pd.read_csv("./DataSources/ABC_for_cluster_0.csv")
cluster_0_label_A_categories = df[df['ABC_Category'] == 'A']['ex3/category'].value_counts().head(10).to_dict()
cluster_0_label_A_categories

{'Groceries': 95,
 'Home & Living': 66,
 'Food & Beverage': 60,
 'Beauty': 36,
 'Fashion Accessories': 28,
 "Women's Shoes and Clothing": 24,
 'Women Clothes': 19,
 'Women Bags': 16,
 'Hobbies & Collections': 16,
 'Household Supplies': 15}

In [22]:
import pandas as pd

# Read the CSV file
df = pd.read_csv("./DataSources/ABC_for_cluster_0.csv")

# Filter data for category 'A' and get the top 10 categories
cluster_0_label_A_categories = df[df['ABC_Category'] == 'A']['ex3/category'].value_counts().head(10).to_dict()

# Create a DataFrame with the required columns
data = {'cluster': [0], 'label': ['A'], 'top_categories': [cluster_0_label_A_categories]}
result_df = pd.DataFrame(data)

# Display the table
print(result_df)

   cluster label                                     top_categories
0        0     A  {'Groceries': 95, 'Home & Living': 66, 'Food &...


In [24]:
import pandas as pd

# Define file paths for cluster data
cluster_file_paths = {
  0: "./DataSources/ABC_for_cluster_0.csv",
  1: "./DataSources/ABC_for_cluster_1.csv",
  2: "./DataSources/ABC_for_cluster_2.csv",
  3: "./DataSources/ABC_for_cluster_3.csv"
}

# Function to process cluster data and generate DataFrame
def process_cluster_data(cluster_data_path):
  # Read data from CSV using pd.read_csv
  cluster_df = pd.read_csv(cluster_data_path)

  # Iterate through unique values in 'ABC_Category' column (assuming labels)
  for label in cluster_df['ABC_Category'].unique():
    cluster_label_categories = (cluster_df[cluster_df['ABC_Category'] == label]
                               ['ex3/category'].value_counts().head(10).to_dict())
    data = {'cluster': [cluster_data_path.split('/')[-1].split('_')[-1][0]], 
             'label': [label], 'top_categories': [cluster_label_categories]}
    yield pd.DataFrame(data)  # Use yield for generator expression

# Generate DataFrames for each cluster with labels
result_dfs = []
for cluster_file_path in cluster_file_paths.values():
  for df in process_cluster_data(cluster_file_path):
    result_dfs.append(df)

# Concatenate DataFrames
result_df = pd.concat(result_dfs)

# Display the table
print(result_df)


  cluster label                                     top_categories
0       0     A  {'Groceries': 95, 'Home & Living': 66, 'Food &...
0       0     B  {'Home & Living': 158, 'Beauty': 69, 'Food & B...
0       0     C  {'Home & Living': 255, 'Beauty': 176, 'Mobile ...
0       1     A  {'Women's Shoes and Clothing': 10, 'Kitchen & ...
0       1     B  {'Motors': 37, 'Mother & Baby': 22, 'Lighting ...
0       1     C  {'Beauty': 42, 'Groceries': 30, 'Household Sup...
0       2     A  {'Mom & Baby': 22, 'Beauty': 13, 'Stationery &...
0       2     B  {'Pets': 29, 'Women's Shoes and Clothing': 28,...
0       2     C  {'Beauty': 33, 'Groceries': 29, 'Health': 19, ...
0       3     A  {'Beauty': 48, 'Pet Supplies': 28, 'Groceries'...
0       3     B  {'Groceries': 58, 'Mother & Baby': 42, 'Beauty...
0       3     C  {'Beauty': 47, 'Home & Living': 47, 'Electroni...


In [29]:
result_df.reset_index(drop=True, inplace=True)
result_df.to_csv("./DataSources/top_category.csv", index=False)